In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import timeit
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from FirstModel import FirstModel
%matplotlib inline

In [2]:
from cs231n.data_utils import load_CIFAR10

def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=10000):
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for the two-layer neural net classifier. These are the same steps as
    we used for the SVM, but condensed to a single function.  
    """
    # Load the raw CIFAR-10 data
    cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'
    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Normalize the data: subtract the mean image
    mean_image = np.mean(X_train, axis=0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image

    return X_train, y_train, X_val, y_val, X_test, y_test


# Invoke the above function to get our data.
X_train, y_train, X_val, y_val, X_test, y_test = get_CIFAR10_data()
# enc = OneHotEncoder()
# y_train = enc.fit_transform(y_train.reshape(-1, 1))
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,)
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [3]:
y_out = FirstModel(32, 32, 3)

In [4]:
try:
    with tf.Session() as sess:
        with tf.device("/gpu:0") as dev: #"/cpu:0" or "/gpu:0"
            tf.global_variables_initializer().run()

            ans = sess.run(y_out,feed_dict={X:x,is_training:True})
            %timeit sess.run(y_out,feed_dict={X:x,is_training:True})
except tf.errors.InvalidArgumentError:
    print("no gpu found, please use Google Cloud if you want GPU acceleration")    
    # rebuild the graph
    # trying to start a GPU throws an exception 
    # and also trashes the original graph
    tf.reset_default_graph()
#     X = tf.placeholder(tf.float32, [None, 32, 32, 3])
#     y = tf.placeholder(tf.int64, [None])
#     is_training = tf.placeholder(tf.bool)
#     y_out = FirstModel(X,y,is_training)
    y_out = FirstModel(32, 32, 3)

no gpu found, please use Google Cloud if you want GPU acceleration


In [5]:
y_out.train(X_train, y_train)

Training
Iteration 0: with minibatch training loss = 2.62 and accuracy of 0.062
Iteration 100: with minibatch training loss = 1.82 and accuracy of 0.38
Iteration 200: with minibatch training loss = 1.7 and accuracy of 0.42
Iteration 300: with minibatch training loss = 1.6 and accuracy of 0.39
Iteration 400: with minibatch training loss = 1.56 and accuracy of 0.47
Iteration 500: with minibatch training loss = 1.47 and accuracy of 0.52
Iteration 600: with minibatch training loss = 1.62 and accuracy of 0.41
Iteration 700: with minibatch training loss = 1.5 and accuracy of 0.47
Epoch 1, Overall loss = 1.62 and accuracy of 0.441
Iteration 800: with minibatch training loss = 1.52 and accuracy of 0.55
Iteration 900: with minibatch training loss = 1.37 and accuracy of 0.53
Iteration 1000: with minibatch training loss = 1.42 and accuracy of 0.58
Iteration 1100: with minibatch training loss = 1.17 and accuracy of 0.64
Iteration 1200: with minibatch training loss = 1.31 and accuracy of 0.61
Itera

In [6]:
print('Validation')
# run_model(sess,y_out,mean_loss,X_val,y_val,1,64)
y_out.evaluate(X_val, y_val)

Validation
Epoch 1, Overall loss = 0.865 and accuracy of 0.736


In [8]:
y_out.evaluate(X_test,y_test)

Epoch 1, Overall loss = 0.881 and accuracy of 0.738
